# ДЗ: Дообучение модели под свою задачу и интеграция с внешними функциями

Будем дообучать модель YandexGPT Lite по методу LoRA

In [44]:
%pip install yandex-cloud-ml-sdk
%pip install datasets
%pip install evaluate
%pip install rouge_score
%pip install kagglehub[pandas-datasets]
%pip install langchain


I0000 00:00:1760797901.438128   13512 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


Note: you may need to restart the kernel to use updated packages.


I0000 00:00:1760797902.411307   13512 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


Note: you may need to restart the kernel to use updated packages.


I0000 00:00:1760797903.135028   13512 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


Note: you may need to restart the kernel to use updated packages.


I0000 00:00:1760797903.878629   13512 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: kagglehub[pandas-datasets]
Note: you may need to restart the kernel to use updated packages.


I0000 00:00:1760797904.650186   13512 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1760797904.824280   13512 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


Note: you may need to restart the kernel to use updated packages.


Поднимаем sdk

In [4]:
import os
from yandex_cloud_ml_sdk import YCloudML


YANDEX_CLOUD_FOLDER = os.getenv("folder_id")
YANDEX_CLOUD_API_KEY = os.getenv("ya_token")

sdk = YCloudML(
    folder_id=YANDEX_CLOUD_FOLDER,
    auth=YANDEX_CLOUD_API_KEY,
)

sdk.setup_default_logging()

print(sdk)

Создаем базовую модель yandexgpt-lite

In [5]:
base_model = sdk.models.completions("yandexgpt-lite")

Скачиваем датасет bond005/russian-child-tales

In [10]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("bond005/russian-child-tales")

file_path = os.path.join(path, "test.csv")

# Загружаем данные в DataFrame
df = pd.read_csv(file_path)

# Вывод первых 5 записей
print("Первые 5 записей датасета:")
display(df.head())



Первые 5 записей датасета:


,query,response
0,"Расскажи, пожалуйста, какие запахи приносит в ...",На мохнатых колких лапах\nЕлка в дом приносит ...
1,Где же мой сверчок?,"Папа работал, \nШуметь запрещал...\nВдруг\nПод..."
2,"Мама, а ты можешь рассказать мне сказку (или и...","Если вам уже лет двадцать,\nВсё равно вы помни..."
3,Можно я спрошу цветочки откуда они взялись?,"Эй, ромашки,\nДайте мне ответ:\nВы откуда,\nЕс..."
4,"Расскажи, пожалуйста, стишок про рыбалку.","От рыбалки мало проку,\nНе клюёт, хоть тресни,..."


Проверяем ответы базовой модели на запросах из датасета

In [28]:
    result = base_model.configure(
        temperature=0.5,
    ).run("Где же мой сверчок?")

    for alternative in result:
        print(alternative.text)


Уточните, пожалуйста, ваш запрос. Что нужно сделать со сверчком? Если вы хотите найти сверчка в доме или квартире, попробуйте прислушаться в тёмное время суток, возможно, вы услышите его стрекотание. Если у вас есть другие вопросы или задачи, пожалуйста, уточните их.


Переводим датасет в формат понятный модели

In [13]:
import json

data_pd = pd.read_csv(os.path.join(path, "train.csv"))

# переводим в нужный формат
#{"request": [{"role": "user", "text": example["act"]}], "response": example["prompt"]}

output_file = "train_data.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
   for _, row in data_pd.iterrows():
       line = json.dumps({
           "request": [{"role": "user", "text": row["query"]}],
           "response": row["response"]
       }, ensure_ascii=False)
       f.write(line + "\n")



Загружаем датасет в облако или получаем id (если уже был загружен)

In [14]:
# Название датасета для поиска
DATASET_NAME = "YandexGPT_LoRA_Tuning_Dataset"

# Ищем существующий датасет по имени
existing_dataset = None
for dataset in sdk.datasets.list():
    if dataset.name == DATASET_NAME:
        existing_dataset = dataset
        break

if existing_dataset:
    # Используем существующий датасет
    dataset_id = existing_dataset.id
    print(f"Найден существующий датасет с ID: {dataset_id}")
else:
    # Создаём и загружаем новый датасет
    dataset_draft = sdk.datasets.draft_from_path(
        task_type="TextToTextGeneration",
        path=output_file,
        upload_format="jsonlines",
        name=DATASET_NAME
    )

    operation = dataset_draft.upload_deferred()
    tuning_dataset = operation.wait()
    dataset_id = tuning_dataset.id
    print(f"Создан новый датасет с ID: {dataset_id}")


[I 2025-10-18 16:42:36 yandex_cloud_ml_sdk._datasets.domain:168] 0 datasets successfully fetched
[I 2025-10-18 16:42:37 yandex_cloud_ml_sdk._datasets.domain:103] New (empty) dataset fdsjgno7i1vqor649q6j of type TextToTextGeneration successfully created
[I 2025-10-18 16:42:37 yandex_cloud_ml_sdk._datasets.dataset:301] Dataset fdsjgno7i1vqor649q6j upload url successfully fetched
[I 2025-10-18 16:42:38 yandex_cloud_ml_sdk._datasets.draft:171] Data from path train_data.jsonl to dataset fdsjgno7i1vqor649q6j successfully uploaded
[I 2025-10-18 16:42:38 yandex_cloud_ml_sdk._datasets.draft:111] Dataset fdsjgno7i1vqor649q6j validation operation fds0gbomgv3a4ap80n4q started
[I 2025-10-18 16:42:38 yandex_cloud_ml_sdk._types.operation:168] Starting Operation<id='fds0gbomgv3a4ap80n4q', result_type=Dataset, endpoint_name=ai-foundation-models> polling with a poll interval 10.000000s and poll timeout 21600.000000s
[I 2025-10-18 16:44:20 yandex_cloud_ml_sdk._types.operation:149] Operation<id='fds0gbomg

Создан новый датасет с ID: fdsjgno7i1vqor649q6j


Запускаем дообучение

In [31]:
# получаем датасет
train_dataset = sdk.datasets.get(dataset_id)

# Название модели для поиска
MODEL_NAME = "YandexGPT_LoRA_Tuned_Model_Tales"

# запускаем обучение
tuned_model = base_model.tune(
    train_dataset,
    name=MODEL_NAME,
    n_samples=1000,
)
print(f"Resulting {tuned_model}")


[I 2025-10-18 17:05:29 yandex_cloud_ml_sdk._datasets.domain:125] Dataset fdsjgno7i1vqor649q6j successfully fetched
[I 2025-10-18 17:05:29 yandex_cloud_ml_sdk._tuning.domain:93] Tuning operation 'ftnh9rj1ai980eqsn9nr' created for model gpt://b1gmgddvdfql2k4f6vk6/yandexgpt-lite/latest with train datasets (('fdsjgno7i1vqor649q6j', 1.0),) and validation datasets ()
[I 2025-10-18 17:05:29 yandex_cloud_ml_sdk._types.operation:168] Starting TuningTask<id='ftnh9rj1ai980eqsn9nr', result_type=GPTModel(...)> polling with a poll interval 60.000000s and poll timeout 259200.000000s
[I 2025-10-18 17:10:30 yandex_cloud_ml_sdk._types.operation:149] TuningTask<id='ftnh9rj1ai980eqsn9nr', result_type=GPTModel(...)> successfully finished
[I 2025-10-18 17:10:30 yandex_cloud_ml_sdk._types.operation:179] TuningTask<id='ftnh9rj1ai980eqsn9nr', result_type=GPTModel(...)> polling finished


Resulting GPTModel(uri=gpt://b1gmgddvdfql2k4f6vk6/yandexgpt-lite/latest@tamrpd80r0rvd5n06rcdh, config=GPTModelConfig(temperature=None, max_tokens=None, reasoning_mode=None, response_format=None, tools=None, parallel_tool_calls=None, tool_choice=None))


Проверяем результат дообученой модели на том же запросе

In [43]:
tuned_model_url = "gpt://b1gmgddvdfql2k4f6vk6/yandexgpt-lite/latest@tamrsmigif8aocaddnds6"

tuned_model = sdk.models.completions(tuned_model_url)

tuned_result = tuned_model.configure(
    temperature=0.5,
).run("Где же мой сверчок?")

for alternative in tuned_result:
    print(alternative.text)


Сверчок за печкой,
Жужжит на печи.
Ты, сверчок, за печкой,
Не жужжи!

А то дедушка
Тебя найдёт
И веником
Тебя прибьёт!


В результате дообучения модель уже понимает что требуется детское стихотворение

Делаем агента и функциональности с помощью langchain tools

In [57]:
from langchain_community.llms import YandexGPT


yandex_gpt_tales = YandexGPT(
    folder_id=YANDEX_CLOUD_FOLDER,
    api_key=YANDEX_CLOUD_API_KEY,
    model_url="gpt://b1gmgddvdfql2k4f6vk6/yandexgpt-lite/latest@tamrsmigif8aocaddnds6",
    temperature=0.5
)


from langchain.agents import Tool

def get_search_results(query: str) -> str:

    search = sdk.search_api.generative(
        site=['https://tykityk.ru'],
        fix_misspell=True,
        enable_nrfm_docs=True,
    )

    search_result = search.run(query)

    return search_result.text

search_tool = Tool(
    name="web_search",
    func=get_search_results,
    description="Ищет информацию на сайте сказок"
)

from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=[search_tool],
    llm=yandex_gpt_tales,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response = agent.run("Где же мой сверчок?")
print(response)




> Entering new AgentExecutor chain...
Question: Где же мой сверчок?
Thought: I need to find out what this phrase means or if it's related to a story or context. I will use a web search to find information.
Action: web_search
Action Input: Где же мой сверчок?
Observation: Возможно, имелся в виду персонаж из сказки А. Н. Толстого «Золотой ключик или Приключения Буратино» — **Говорящий Сверчок**. [1] 

В произведении сверчок — существо, похожее на таракана, с головой кузнечика, которое живёт в комнате больше ста лет. [1]
Thought:I now know the final answer.

Final Answer: Вероятно, имеется в виду Говорящий Сверчок из сказки А. Н. Толстого «Золотой ключик или Приключения Буратино».

> Finished chain.
Вероятно, имеется в виду Говорящий Сверчок из сказки А. Н. Толстого «Золотой ключик или Приключения Буратино».
